# Generation of Prompt Summaries with CarperAI Summarization Model

## Preparations

In [1]:
#!pip install torch torchvision torchaudio pandas

In [3]:
# imports !!! am ende nochmal durchschauen, welche wirklich gebraucht werden !!!

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm
import time

from datasets import load_dataset
import matplotlib.pyplot as plt
#import spacy
#import eng_spacysentiment
import numpy as np
import pandas as pd
from collections import Counter
from ast import literal_eval
from scipy.stats import wilcoxon

## Load Matching and Mismatching Prompts

In [ ]:
# load matches and mismatches
match_prompts = pd.read_csv("match_prompts.csv")
mismatch_prompts = pd.read_csv("mismatch_prompts.csv")

In [ ]:
# rename columns in the way polina needs them
match_prompts = match_prompts.rename(columns={'Unnamed: 0': 'prompt_id', 'prompt': 'text'})
mismatch_prompts = mismatch_prompts.rename(columns={'Unnamed: 0': 'prompt_id', 'prompt': 'text'})

## Predict summarizations by feeding selected prompts in summarization model

In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("CarperAI/openai_summarize_tldr_ppo")

In [ ]:

def generate_summaries(prompts_df):
    
    # List to store the summaries
    summaries = []

    # Iterate over the rows in the DataFrame
    for text in tqdm(prompts_df['text'], desc="Generating summaries"):
        
        # Tokenize the text input for the model
        inputs = tokenizer(text, return_tensors="pt")
        
        # Generate the summary output tokens
        summary_tokens = model.generate(inputs['input_ids'], max_length=1000,do_sample=False)
        
        
        # Decode the tokens to a string and append to the list
        summary = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)
        summaries.append(summary)

    # Add the summaries to the DataFrame
    prompts_df['summary'] = summaries
    return prompts_df

In [ ]:
# generate summaries for matching and mismatching test prompts
summaries_df_match = generate_summaries(match_prompts)
summaries_df_mismatch = generate_summaries(mismatch_prompts)

In [ ]:
# concatenate matching and mismatching summary datasets
summaries_df = pd.concat([summaries_df_match, summaries_df_mismatch], ignore_index=True)
summaries_df

In [ ]:
summaries_df['summaryOnly'] = summaries_df['summaryOnly'].apply(lambda x: x.split("DR:")[1])

In [ ]:
summaries_df.rename(columns={'summary': 'Outputs'}, inplace=True)

In [ ]:
# save df in csv
summaries_df.to_csv("text_summaries.csv")